In [1]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# 🛠️ **Custom Prompt Templates in LangChain:**

- ✂️ **Tailor-Made**: Craft prompts that fit your model like a glove.

- 🎨**Full Control**: Dictate every detail from instructions to formatting.

- 🔌 **Dynamic Inputs**: Plug in specific data as needed for each task.

Create your own by defining inputs and crafting a `format()` method. 📝

📐 Custom templates are like a bespoke suit—made to fit your model's unique requirements.


#🎨 **Custom vs. Default Prompt Templates:**

- ✏️ **Default Templates**: Like using a template for a letter, where you just fill in the blanks.

- 🎭 **Custom Templates**: Like writing a script for a play, where you have the freedom to craft the entire scene.


In [3]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

template = PromptTemplate.from_template("Hello {name}!")

prompt = template.format(name="Harpreet")

print(prompt)

Hello Harpreet!



**Custom templates let you:**

- 🛠️ **Programmatically Craft Prompts**: Build prompts on-the-fly, tailored to the task at hand.

- **Example - `FunctionExplainerPromptTemplate`**:

  - 👨🏽‍💻 **Function Input**: Takes a function directly.

  - 🔍 **Code Inspection**: Uses `inspect` to get the function's source code.

  - 🗣️ **Prompt Assembly**: Creates a prompt to explain the function in plain language.

🗝️ This approach is like having a Swiss Army knife for prompt creation, giving you the tools to construct exactly what you need.


In [8]:
# import inspect
# from langchain.prompts import StringPromptTemplate
# from pydantic import BaseModel, validator

# def get_source_code(function_name):
#     """Return the source code of the provided function."""
#     # Using the inspect module to get the source code of the function
#     return inspect.getsource(function_name)

# # Template string for the prompt that will be sent to the language model
# PROMPT = """Given the function name and source code, generate an English language explanation of the function.
# Function Name: {function_name}

# Source Code:
# {source_code}

# Explanation:
# """

# class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
#     """A custom prompt template that takes in the function name as input and formats the prompt template to provide the source code of the function."""

#     @validator("input_variables")
#     def validate_input_variables(cls, v):
#         """Validate that the input variables are correct."""
#         # Ensuring that the only input variable is 'function_name'
#         if len(v) != 1 or "function_name" not in v:
#             raise ValueError("function_name must be the only input_variable.")
#         return v

#     def format(self, **kwargs) -> str:
#         """Format the prompt using the function's name and source code."""
#         # Retrieve the source code of the provided function
#         source_code = get_source_code(kwargs["function_name"])

#         # Format the PROMPT string using the function name and its source code
#         prompt = PROMPT.format(
#             function_name=kwargs["function_name"].__name__,
#             source_code=source_code
#         )
#         return prompt

#     def _prompt_type(self) -> str:
#         """Return the type of prompt."""
#         return "function-explainer"

In [15]:
from langchain.prompts import StringPromptTemplate

# Template string for the prompt that will be sent to the language model
PROMPT = """Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}

Source Code:
{source_code}

Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate):
    """A custom prompt template that takes in the function name as input and formats the prompt template to provide the source code of the function."""

    def format(self, **kwargs) -> str:
        """Format the prompt using the function's name and source code."""
        # Retrieve the source code of the provided function
        source_code = inspect.getsource(kwargs["function_name"])

        # Format the PROMPT string using the function name and its source code
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "function-explainer"

In [16]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])


In [17]:
type(fn_explainer)

__main__.FunctionExplainerPromptTemplate

In [18]:
# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)

print(prompt)

Given the function name and source code, generate an English language explanation of the function.
Function Name: get_source_code

Source Code:
def get_source_code(function_name):
    """Return the source code of the provided function."""
    # Using the inspect module to get the source code of the function
    return inspect.getsource(function_name)


Explanation:



In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

response = llm.invoke(prompt)

response.content

'This function called get_source_code takes in a parameter called function_name. It uses the inspect module to fetch and return the source code of the function that is provided as an argument.'

In [20]:
from langchain_core.output_parsers import StrOutputParser

llm_chaim = fn_explainer | llm | StrOutputParser()

for chunk in llm_chaim.stream({"function_name":get_source_code}):
  print(chunk, end="", flush=True)

This function, named "get_source_code", takes a function name as input and returns the source code of the provided function. It uses the inspect module to retrieve the source code of the function and then returns it as a string. This can be useful for analyzing or manipulating the source code of a function programmatically.

# Another example


In [ ]:
# Template string for the algorithm optimization prompt
ALGO_PROMPT = """Given the algorithm below, suggest optimizations or potential \
improvements, and return the optimized code.

Algorithm Name: {algorithm_name}

Source Code:
{source_code}

Suggestions:
"""

class AlgorithmOptimizerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes an algorithm as input and formats the prompt template to request optimization suggestions."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "algorithm_function" not in v:
            raise ValueError("algorithm_function must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        """Format the prompt using the algorithm's name and source code."""

        # Retrieve the source code of the provided algorithm
        source_code = get_source_code(kwargs["algorithm_function"])

        # Format the ALGO_PROMPT string using the algorithm name and its source code
        prompt = ALGO_PROMPT.format(
            algorithm_name=kwargs["algorithm_function"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "algorithm-optimizer"


In [ ]:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion."""
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


In [ ]:
# Instantiate the AlgorithmOptimizerPromptTemplate with the appropriate input variable
algo_optimizer = AlgorithmOptimizerPromptTemplate(input_variables=["algorithm_function"])

# Generate a prompt for the function "recursive_factorial"
prompt = algo_optimizer.format(algorithm_function=recursive_factorial)

print(prompt)

In [ ]:
result = llm.invoke(prompt)

print(result.content)

In [ ]:
llm_chain = algo_optimizer | llm | StrOutputParser()

for chunk in llm_chain.stream({"algorithm_function":recursive_factorial}):
  print(chunk, end="", flush=True)